In [1]:
import math
math.radians(90)

1.5707963267948966

In [9]:
import mujoco

# Load the MuJoCo model from the XML file
model = mujoco.MjModel.from_xml_path('left_hand_light.xml')



In [10]:
num_camera=model.ncam
print(num_camera)


2


In [5]:
import xml.etree.ElementTree as ET


In [6]:

def get_cameras_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    cameras = []

    # Iterate over all camera elements in the XML
    for camera in root.findall('.//camera'):
        cam_name = camera.get('name')
        cam_pos = camera.get('pos')
        cam_quat = camera.get('quat')

        # Convert the position and quaternion strings to lists of floats
        if cam_pos:
            cam_pos = list(map(float, cam_pos.split()))
        if cam_quat:
            cam_quat = list(map(float, cam_quat.split()))

        cameras.append({
            'name': cam_name,
            'pos': cam_pos,
            'quat': cam_quat
        })

    return cameras

In [8]:
# Get camera information
cameras = get_cameras_from_xml("left_hand_light.xml")

# Print camera information
for cam in cameras:
    print(f"Camera Name: {cam['name']}")
    print(f"Position: {cam['pos']}")
    print(f"Orientation (Quaternion): {cam['quat']}")
    print()

Camera Name: cam1
Position: [0.0, 0.0, 2.0]
Orientation (Quaternion): [1.0, 0.0, 0.0, 0.0]

Camera Name: cam2
Position: [2.0, 0.0, 0.0]
Orientation (Quaternion): [0.707, 0.0, 0.707, 0.0]



In [2]:
# Get the number of joints
num_joints = model.njnt
print(f"Number of joints: {num_joints}")

Number of joints: 16


In [17]:
# Get the joint names, types, and orientations in space
joint_info = []
for i in range(num_joints):
    joint_name = model.name_jntadr[i]
    joint_body = model.jnt_bodyid[i]   
    
    joint_type = model.jnt_type[i]
    # Orientation in space: Joint axis or orientation quaternion (depends on joint type)
    joint_axis = model.jnt_axis[i] 
    joint_info.append((joint_name, joint_type, joint_body,joint_axis))
# print(len(joint_name))
print(joint_info)

[(212, 3, 2, array([0., 0., 1.])), (217, 3, 3, array([0., 1., 0.])), (222, 3, 4, array([0., 1., 0.])), (227, 3, 5, array([0., 1., 0.])), (232, 3, 7, array([0., 0., 1.])), (237, 3, 8, array([0., 1., 0.])), (242, 3, 9, array([0., 1., 0.])), (247, 3, 10, array([0., 1., 0.])), (252, 3, 12, array([0., 0., 1.])), (257, 3, 13, array([0., 1., 0.])), (262, 3, 14, array([0., 1., 0.])), (267, 3, 15, array([0., 1., 0.])), (272, 3, 17, array([1., 0., 0.])), (277, 3, 18, array([ 0.,  0., -1.])), (282, 3, 19, array([0., 1., 0.])), (287, 3, 20, array([0., 1., 0.]))]


In [33]:
joint_name = model.names
print(joint_name)


b'allegro_left\x00world\x00palm\x00rf_base\x00rf_proximal\x00rf_medial\x00rf_distal\x00rf_tip\x00mf_base\x00mf_proximal\x00mf_medial\x00mf_distal\x00mf_tip\x00ff_base\x00ff_proximal\x00ff_medial\x00ff_distal\x00ff_tip\x00th_base\x00th_proximal\x00th_medial\x00th_distal\x00th_tip\x00rfj0\x00rfj1\x00rfj2\x00rfj3\x00mfj0\x00mfj1\x00mfj2\x00mfj3\x00ffj0\x00ffj1\x00ffj2\x00ffj3\x00thj0\x00thj1\x00thj2\x00thj3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00base_link_left\x00link_0.0\x00link_1.0\x00link_2.0\x00link_3.0\x00link_3.0_tip\x00link_12.0_left\x00link_13.0\x00link_14.0\x00link_15.0\x00link_15.0_tip\x00black\x00white\x00\x00\x00\x00\x00\x00ffa0\x00ffa1\x00ffa2\x00ffa3\x00mfa0\x00mfa1\x00mfa2\x00mfa3\x00rfa0\x00rfa1\x00rfa2\x00rfa3\x00tha0\x00tha1\x00tha2\x00tha3\x00'


In [ ]:

# Get the number of joints
num_joints = model.njnt
print(f"Number of joints: {num_joints}")

# Get the joint names, types, and orientations in space
joint_info = []
for i in range(num_joints):
    joint_name = model.joint_id2name(i)
    joint_type = model.jnt_type[i]
    # Orientation in space: Joint axis or orientation quaternion (depends on joint type)
    joint_axis_or_quat = model.jnt_axis[i] if joint_type in [2, 3] else model.jnt_quat[i]
    joint_info.append((joint_name, joint_type, joint_axis_or_quat))

# Map joint types to human-readable strings
joint_type_mapping = {
    0: 'free',
    1: 'ball',
    2: 'slide',
    3: 'hinge',
    4: 'plane',
    5: 'fixed'
}

# Print joint information
print("Joint Information:")
for name, jtype, axis_or_quat in joint_info:
    readable_type = joint_type_mapping.get(jtype, 'unknown')
    print(f"Name: {name}, Type: {readable_type}, Orientation: {axis_or_quat}")

# Get the number of bodies
num_bodies = model.nbody
print(f"Number of bodies: {num_bodies}")

# Get the body names
body_names = []
for i in range(num_bodies):
    body_name = model.body_id2name(i)
    body_names.append(body_name)

# Print body names
print("Body Names:")
for name in body_names:
    print(name)

In [25]:

# Get the number of bodies
num_bodies = model.nbody
print(f"Number of bodies: {num_bodies}")



# Get the number of bodies
num_site = model.nsite
print(f"Number of site: {num_site}")
n_cam=model.ncam
print(n_cam)


Number of bodies: 22
Number of site: 0
0


In [29]:
# Get the body names
body_names = []
for i in range(num_bodies):
    body_name = model.name_bodyadr[i]
    body_names.append(body_name)


body_names_parent = []
for i in range(num_bodies):
    body_name = model.body_weldid[i]
    body_names_parent.append(body_name)
# Print body names
print(len(body_names))
print("Body Names Parent:")

for name in body_names_parent:
    print(name)

22
Body Names Parent:
0
0
2
3
4
5
5
7
8
9
10
10
12
13
14
15
15
17
18
19
20
20


In [34]:
import mujoco.viewer